<a href="https://colab.research.google.com/github/0xZee/llama-index-rag/blob/main/llama_index_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL & LIB
---

In [12]:
!pip install -q llama-index llama-index-llms-groq llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.4 MB/s eta 0:00:00


In [13]:
from google.colab import userdata
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
#from llama_index.experimental.query_engine import PandasQueryEngine

from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#from llama_index.llms.cohere import Cohere
#from llama_index.embeddings.cohere import CohereEmbedding

LLM / EMBED SETTING
---

In [14]:
# Setting LLM
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
model_llama = "llama3-8b-8192"
model_gemma = "gemma-7b-it"
model_mixtral = "mixtral-8x7b-32768"
model_whistper = "whisper-large-v3"

LLM_GROQ = Groq(model=model_llama, api_key=GROQ_API_KEY)
Settings.llm = LLM_GROQ

# Setting Embeddings
#EMBED_CO = CohereEmbedding(cohere_api_key=COHERE_API, model_name="embed-english-v3.0", input_type="search_query",)
EMBED_HF = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = EMBED_HF

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
embeddings = EMBED_HF.get_text_embedding("Hello world")
print (len(embeddings))
print(embeddings[:5])

384
[0.015196100808680058, -0.022570667788386345, 0.0085471011698246, -0.07417059689760208, 0.0038364154752343893]


DATA and VECTORSTORE
---

In [21]:
data = SimpleDirectoryReader(input_dir="./").load_data()


In [22]:
index = VectorStoreIndex.from_documents(data)

RAG
---

In [23]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

In [26]:
question = "what the doc is about ?"
response = chat_engine.chat(question)


Querying with: Here is a rewritten version of the follow-up message as a standalone question that captures all relevant context from the conversation:

"What is the purpose or topic of the document?"

This rewritten question takes into account the context of the original conversation, where the user asked about the DOS, and the assistant responded that the DOS was not mentioned in the provided context. The rewritten question assumes that the user is still referring to the same document or topic, and asks about its purpose or topic.


In [27]:
print(response)

The purpose or topic of the document appears to be the author's personal and professional journey, including his experiences with Lisp, Arc, and Bel, as well as his thoughts on entrepreneurship, writing, and the evolution of the internet.


In [28]:
response = chat_engine.chat("and what else ?")

Querying with: Here is a rewritten standalone question that captures all relevant context from the conversation:

"What else is discussed in the document about the author's personal and professional journey, including their experiences with Lisp, Arc, and Bel, as well as their thoughts on entrepreneurship, writing, and the evolution of the internet?"


In [29]:
print(response)

The author's personal and professional journey is discussed throughout the document, covering their experiences with Lisp, Arc, and Bel. They share their thoughts on entrepreneurship, writing, and the evolution of the internet.


In [30]:
response = chat_engine.chat("can you tell me more ?")

Querying with: Here is a rewritten standalone question that captures all relevant context from the conversation:

"What else can you tell me about the author's personal and professional journey, including their experiences with Lisp, Arc, and Bel, as well as their thoughts on entrepreneurship, writing, and the evolution of the internet?"


In [31]:
print(response)

The author's personal and professional journey is marked by a series of unconventional choices and experiences that have shaped their perspective on entrepreneurship, writing, and the evolution of the internet. One of the most significant events in their journey was the development of Lisp, Arc, and Bel, three programming languages that have allowed them to explore the intersection of technology and art.

The author's experience with Lisp, a programming language developed in the 1950s, was instrumental in their understanding of the potential of computers to transform industries. Their work on Arc, a dialect of Lisp, further solidified their expertise in the field and allowed them to apply their knowledge to real-world problems.

The development of Bel, a new Lisp, was a particularly challenging and rewarding experience for the author. After four years of intense focus, they finally completed the project, which they describe as a "spec rather than an implementation." The experience taug

In [32]:
# RESET CHAT SESSION
chat_engine.reset()

In [36]:
# STREAMING

response = chat_engine.stream_chat("can you summarize the doc for me in 6 bullets ?")
for token in response.response_gen:
    print(token, end="")

Querying with: Here is a rewritten standalone question that captures all relevant context from the conversation:

"What were the key events in Paul Graham's life after he stopped working on Y Combinator, including his transition to painting, his eventual return to writing and Lisp development, and any notable milestones or realizations he had during this time?"
After stopping work on Y Combinator, Paul Graham started painting, focusing on it for most of 2014. He then shifted his attention to writing essays and eventually returned to Lisp development. A notable milestone was his decision to write a book about Lisp hacking, which became "On Lisp". He also started working on a new Lisp implementation, called Bel, which took four years to complete.